In [12]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from res_ind_lib import *

#Imports the function to plot render the policy experiments
from policy_assessment import *
from apply_policy import apply_policy

#ask jupyter to display the plots within this notebook
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Baseline data and function

## parameters

In [13]:
economy = "country"
event_level = [economy, "hazard", "rp"]

## inputs

In [14]:
#Load the excel data file into a dataframe (table)
macro=pd.read_csv("intermediate/macro.csv", 
                 index_col=economy, #tells Pandas to use the first column (Provinces) as the index of the table
                ).dropna() 


# macro=def_ref_values(macro)

#Displays the first rows of that dataframe
macro.head()

macro["gdp_pc_pp_ref"] =macro["gdp_pc_pp"] 



In [15]:
cat_info = pd.read_csv("intermediate/cat_info.csv", index_col=[economy,"income_cat"])

# cat_info.v = cat_info.v.unstack().assign(poor=lambda df:df.nonpoor).stack()


cat_info.head()

n             c     axfin  gamma_SP             k  \
country   income_cat                                                        
Albania   nonpoor     0.8  11349.246839  0.092387  1.094243  61811.780436   
          poor        0.2   4407.716271  0.075853  0.623028  21560.668539   
Angola    nonpoor     0.8   8854.573906  0.201269  1.160476  20523.147061   
          poor        0.2   2021.763175  0.091791  0.358097   4494.564269   
Argentina nonpoor     0.8  21573.503341  0.060371  1.171127  80282.025353   

                            fa         v  shew  
country   income_cat                            
Albania   nonpoor     0.022104  0.351500  0.00  
          poor        0.021223  0.700000  0.00  
Angola    nonpoor     0.002814  0.600000  0.00  
          poor        0.012449  0.700000  0.00  
Argentina nonpoor     0.008573  0.397254  0.64

In [16]:
hazard_ratios = pd.read_csv("intermediate/hazard_ratios.csv", index_col=event_level+["income_cat"])

hazard_ratios.head()

fa  shew
country     hazard     rp   income_cat                
Afghanistan earthquake 1.0  nonpoor     0.004196   0.0
                            poor        0.004196   0.0
                       20.0 nonpoor     0.011053   0.0
                            poor        0.011053   0.0
                       50.0 nonpoor     0.022173   0.0

In [17]:
print("macro variables are: "+", ".join(macro.columns))
print("micro variables are: "+", ".join(cat_info.columns))
print("hazard variables are: "+", ".join(hazard_ratios.columns))

macro variables are: gdp_pc_pp, pop, urbanization_rate, prepare_scaleup, borrow_abi, avg_prod_k, T_rebuild_K, pi, income_elast, rho, shareable, max_increased_spending, fa_glofris, protection, tau_tax, gdp_pc_pp_ref
micro variables are: n, c, axfin, gamma_SP, k, fa, v, shew
hazard variables are: fa, shew


## output

In [18]:
refout = compute_resilience(macro,cat_info,hazard_ratios=hazard_ratios,verbose_replace=True)
# .drop("Togo",axis=0)

Replaced in macro: 
Replaced in cats: fa, shew
Replaced in both: 


In [19]:
refout.dKtot.sum()*1e-9 , refout.dWtot_currency.sum()*1e-9

(327.45825854332935, 517.3058432735279)

## function test_package

In [20]:
def test_package(m,c,h,a,return_rel=False,macro=macro,hazard_ratios=hazard_ratios,cat_info=cat_info,refout=refout, verbose=True):
    """test a policy package where:
    -macro, cat_info and hazard_ratios, which are assumed to yield refout (all 4 optional, defautlt o baseline)
    are replaced by:
    m,c,and h. 
    This function states the difference between old and new set of params, runs the model with new set of paramters, 
    and reports difference in dW and dk"""
        
    #State differences
    if verbose:
        print("Changed in macro: "+changed_lines(m,macro))
        print("Changed in cat_info: "+changed_lines(c,cat_info))
        print("Changed in hazard: "+changed_lines(h,hazard_ratios))
        print("args: "+repr(a))

    #New outputs from new inputs
    nu = compute_resilience(m,c,h,**a)

    #absolute and relative difference
    delta =  refout-nu
    rel = delta/refout
    reltogdp = (delta.T/(m["gdp_pc_pp"]*m["pop"])).T
    
    #sumarizes difference in outputs
    if verbose:
        print("\nAsset losses were reduced by {dk:.1f} bn$ ({dkr:.1%} or {g:.3%} of WDP)".format(
                dk=delta.dKtot.sum()*1e-9,
                dkr=delta.dKtot.sum()/refout.dKtot.sum(),
                g = delta.dKtot.sum()/(m["gdp_pc_pp"]*m["pop"]).sum())
             )
        print("Welfare losses were reduced by {dw:.2f} bn$ ({dwr:.2%}  or {g:.3%} of WDP)".format(
                dw=delta.dWtot_currency.sum()*1e-9,
                dwr=delta.dWtot_currency.sum()/refout.dWtot_currency.sum(),
                g = delta.dWtot_currency.sum()/(m["gdp_pc_pp"]*m["pop"]).sum())
             )
    
        print("Resilience in countries changed by {dr:.2%}, from ({old:.2%} to {new:.2%} )".format(
                dr=delta.resilience.mean(),
                old=refout.resilience.mean(),
                new =nu.resilience.mean()             )
              )
        
        old=refout.dKtot.sum()/refout.dWtot_currency.sum()
        new =nu.dKtot.sum()/nu.dWtot_currency.sum()     
        print("Resilience GLOBALLY changed {d:.2%}, from ({old:.2%} to {new:.2%} )".format(
                    d=new-old,
            old=old,
            new=new)
              )
    
    
    
    #returns rel and abs difference
    if return_rel:
        return delta,rel,reltogdp,nu[["risk_to_assets","resilience","risk"]]
    
    return delta
    

# Compute policy card 

In [21]:
policy_list = ["prepare_scaleup", "borrow_abi",  "T_rebuild_K", "social_p","fap","far", "vp","kp","vr", "shew", 
               "axfin","PDSpackage","ResiliencePackage","ResiliencePlusEW","Asset_losses","Asset_losses_no_EW", "prop_nonpoor", "prop_nonpoor_lms"]

# policy_list = ["PDSpackage","ResiliencePackage","ResiliencePlusEW", "prop_nonpoor", "prop_nonpoor_lms"]



# policy_list = ["ResiliencePlusEW"]

country_list = macro.dropna().index.tolist()


pol_assess_set = ["dKtot","dWtot_currency"]

In [22]:

#initialize input (baseline inputs)
m_=macro.copy(deep=True)
c_=cat_info.copy(deep=True)
hr_ = hazard_ratios.copy(deep=True)
a_=dict(verbose_replace=True)

#initialize outptus
delta = pd.DataFrame(index=pd.MultiIndex.from_product([country_list, pol_assess_set+["risk_to_assets","resilience","risk"]], names=[economy, 'outputs']), 
                     columns=pd.Index([],name="policies"))
rel = delta.copy()
reltogdp=  delta.copy()
nuvalues=delta.copy()
policy_description = pd.Series(index=policy_list)

#loop over policies
for pol in policy_list:
    print("\n===\npolicy: "+pol)
    m,c,hr,a,desc =apply_policy(m_,c_,hr_,a_ ,policy_name=pol)
    
    d,r,rto,nv=test_package(m,c,hr,a,return_rel=True)
    
    delta[pol]= d[pol_assess_set].stack()
    rel[pol]  = r[pol_assess_set].stack()
    reltogdp[pol] = rto[pol_assess_set].stack()
    nuvalues[pol] = nv.stack()
    
    
    policy_description[pol] = desc
    
#format as renders wants it    
delta =delta.dropna().unstack()
rel =rel.dropna().unstack()
reltogdp =reltogdp.dropna().unstack()


===
policy: prepare_scaleup
Make social protection scalable after a shock
Changed in macro: prepare_scaleup (~67.9%).
Changed in cat_info: none.
Changed in hazard: none.
args: {'verbose_replace': True}
Replaced in macro: 
Replaced in cats: fa, shew
Replaced in both: 

Asset losses were reduced by -0.0 bn$ (-0.0% or -0.000% of WDP)
Welfare losses were reduced by 4.82 bn$ (0.93%  or 0.005% of WDP)
Resilience in countries changed by -0.52%, from (61.06% to 61.58% )
Resilience GLOBALLY changed 0.59%, from (63.30% to 63.90% )

===
policy: borrow_abi
Develop contingent finance and reserve funds
Changed in macro: borrow_abi (~73.3%).
Changed in cat_info: none.
Changed in hazard: none.
args: {'verbose_replace': True}
Replaced in macro: 
Replaced in cats: fa, shew
Replaced in both: 

Asset losses were reduced by 0.0 bn$ (0.0% or 0.000% of WDP)
Welfare losses were reduced by 4.54 bn$ (0.88%  or 0.005% of WDP)
Resilience in countries changed by -0.62%, from (61.06% to 61.68% )
Resilience GLOBALL

In [23]:
a=pd.read_csv("intermediate/main_results.csv", index_col="country")
a.head()

big_table=pd.DataFrame(index=pd.MultiIndex.from_product([country_list, a.columns | delta.ResiliencePlusEW.columns ]))
big_table["Current"] = a.stack()

big_table["Difference"] = delta.ResiliencePlusEW.stack()

big_table["Differencerel"] =  rel.ResiliencePlusEW.stack()

big_table["With_package"] = nuvalues.ResiliencePlusEW

big_table.unstack().to_excel("results/big_table.xlsx")


big_table.Difference.unstack().describe()


C:\Users\Admin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,dKtot,dWtot_currency,fa,fa_in_gdp,gdp_pc_pp,pop,resilience,risk,risk_to_assets,v
count,1.170000e+02,1.170000e+02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.133746e+08,8.176737e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,3.264548e+08,1.765241e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-1.907349e-06,1.862957e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2.298860e+09,1.070681e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Export excel

In [24]:
from os import makedirs
from fancy_round import fancy_round

In [25]:
makedirs("cards", exist_ok=True)

In [26]:
with pd.ExcelWriter('cards/policy_impact.xlsx') as writer:
        delta.rename(columns=policy_description).rename(columns={"dKtot":"Asset losses", "dWtot_currency":"Wellbeing losses"}
                                                       ).to_excel(writer, sheet_name="absolute (USD PPP)")

        rel.rename(columns=policy_description).rename(columns={"dKtot":"Asset losses", "dWtot_currency":"Wellbeing losses"}
                                                       ).to_excel(writer, sheet_name="relative to current losses")


In [27]:
delta.head(1)

policies prepare_scaleup                borrow_abi                T_rebuild_K  \
outputs            dKtot dWtot_currency      dKtot dWtot_currency       dKtot   
country                                                                         
Albania              0.0   3.683971e+06        0.0            0.0         0.0   

policies                social_p                          fap                 \
outputs  dWtot_currency    dKtot dWtot_currency         dKtot dWtot_currency   
country                                                                        
Albania    1.536462e+07      0.0   1.526867e+07  1.052543e+07   4.015340e+07   

policies      ...       ResiliencePlusEW                 Asset_losses  \
outputs       ...                  dKtot dWtot_currency         dKtot   
country       ...                                                       
Albania       ...           2.129564e+07   9.249498e+07  5.046550e+07   

policies                Asset_losses_no_EW                prop_nonpoor  \
outputs  dWtot_currency              dKtot dWtot_currency        dKtot   
country                                                                  
Albania    9.710229e+07       3.151070e+07   7.009277e+07          0.0   

policies                prop_nonpoor_lms                 
outputs  dWtot_currency            dKtot dWtot_currency  
country                                                  
Albania    1.746798e+07              0.0   1.008888e+07  

[1 rows x 36 columns]

In [28]:
for c in delta.index:
    x=pd.concat([1e-6*delta.ix[c].unstack()[["dWtot_currency","dKtot"]].rename(columns={"dKtot":"Asset losses m$", "dWtot_currency":"Wellbeing losses m$"}),
           100*rel.ix[c].unstack()[["dWtot_currency","dKtot"]].rename(columns={"dKtot":"Asset losses %", "dWtot_currency":"Wellbeing losses %"})],axis=1).sort_values(by="Wellbeing losses m$", 
           ascending=False).applymap(fancy_round).drop(["ResiliencePlusEW", "Asset_losses", "Asset_losses_no_EW", "ResiliencePackage","prop_nonpoor_lms","prepare_scaleup","borrow_abi"]
                                                      ).rename(index=policy_description)
    x.index.name=c
    x.to_csv("cards/country_tables/"+c+".csv"
            )

In [29]:
x

outputs,Wellbeing losses m$,Asset losses m$,Wellbeing losses %,Asset losses %
Zambia,,,,
Increase social\ntransfers to poor\npeople to at\nleast 33%,40.0,0.00,15.00,0.00
Accelerate\nreconstruction\n(by 33%),25.0,0.00,9.20,0.00
Reduce exposure\nof the poor by 5%\nof total exposure,24.0,1.60,8.80,1.50
Postdisaster\nsupport\npackage,21.0,0.00,7.60,0.00
Universal\naccess to\nfinance,15.0,0.00,5.60,0.00
Reduce asset\nvulnerability\n(by 30%) of\npoor people\n(5% of the population),13.0,0.78,4.70,0.71
Reduce\nexposure of the nonpoor by 5%\nof total exposure,9.3,7.20,3.40,6.60
Universal\naccess to early\nwarnings,9.3,3.00,3.40,2.80
Develop market\ninsurance\n(nonpoor people),5.5,0.00,2.00,0.00


In [49]:
def f(x):
    s="{s}".format(s=fancy_round(x,2))
    if s.endswith(".0"):
        s=s[:-2]
    if s=="-0":
        s="0"
    return s    

In [31]:
for c in delta.index:

    y=1e-6*delta.ix[c].unstack()[["dWtot_currency","dKtot"]]
    x=(y.applymap(f).astype(str) 
     + ' ('+(100*rel.ix[c].unstack()[["dWtot_currency","dKtot"]]).applymap(f).astype(str)+'%)'
    )


    x=x.rename(columns={"dKtot":"Asset losses label", "dWtot_currency":"Wellbeing losses  label"}
            )

    y=y.rename(columns={"dKtot":"Asset losses value", "dWtot_currency":"Wellbeing losses value"}
            )

    idx = delta.ix[c].unstack().sort_values(by="dWtot_currency", 
              ascending=False).drop(["ResiliencePlusEW", "Asset_losses", "Asset_losses_no_EW", "ResiliencePackage","prop_nonpoor_lms","prepare_scaleup","borrow_abi"]).index


    # x=
    # x
    t = pd.concat([y,x],axis=1).ix[idx].rename(index=policy_description)
    t.index.name=c
    t.to_csv("cards/country_tables/"+c+".csv")

In [32]:
t

outputs,Wellbeing losses value,Asset losses value,Wellbeing losses label,Asset losses label
Zambia,,,,
Increase social\ntransfers to poor\npeople to at\nleast 33%,40.118880,0.000000,40 (15%),0 (0%)
Accelerate\nreconstruction\n(by 33%),25.358857,0.000000,25 (9.2%),0 (0%)
Reduce exposure\nof the poor by 5%\nof total exposure,24.259087,1.628743,24 (8.8%),1.6 (1.5%)
Postdisaster\nsupport\npackage,20.866879,0.000000,21 (7.6%),0 (0%)
Universal\naccess to\nfinance,15.433415,0.000000,15 (5.6%),0 (0%)
Reduce asset\nvulnerability\n(by 30%) of\npoor people\n(5% of the population),12.946793,0.775134,13 (4.7%),0.78 (0.71%)
Reduce\nexposure of the nonpoor by 5%\nof total exposure,9.308886,7.240204,9.3 (3.4%),7.2 (6.6%)
Universal\naccess to early\nwarnings,9.272713,3.004406,9.3 (3.4%),3 (2.8%)
Develop market\ninsurance\n(nonpoor people),5.489319,0.000000,5.5 (2%),0 (0%)


## policy tables for Brad

In [37]:
delta.stack().columns

Index(['prepare_scaleup', 'borrow_abi', 'T_rebuild_K', 'social_p', 'fap',
       'far', 'vp', 'kp', 'vr', 'shew', 'axfin', 'PDSpackage',
       'ResiliencePackage', 'ResiliencePlusEW', 'Asset_losses',
       'Asset_losses_no_EW', 'prop_nonpoor', 'prop_nonpoor_lms'],
      dtype='object', name='policies')

In [50]:
makedirs("cards/policy_tables/", exist_ok=True)

for col in delta.stack().columns:
    y=1e-6*delta[col]
    x=(y.applymap(f).astype(str) 
     + ' ('+(100*rel[col][["dWtot_currency","dKtot"]]).applymap(f).astype(str)+'%)'
    )


    x=x.rename(columns={"dKtot":"Asset losses label", "dWtot_currency":"Wellbeing losses  label"}
            )

    x


    y=y.rename(columns={"dKtot":"Asset losses value", "dWtot_currency":"Wellbeing losses value"}
                )

    idx = delta[col].sort_values(by="dWtot_currency", 
              ascending=False).index


    t = pd.concat([y,x],axis=1).ix[idx]
    t.index.name=policy_description[col]

    t.to_csv("cards/policy_tables/"+col+".csv");
t.head(15)

outputs,Asset losses value,Wellbeing losses value,Asset losses label,Wellbeing losses label
"Develop market insurance (25% of population, only nonpoor)",,,,
China,0.000000e+00,1957.303255,0 (0%),2000 (2.6%)
United States,0.000000e+00,1195.193845,0 (0%),1200 (2%)
Japan,-3.814697e-12,876.848735,0 (0%),880 (2.3%)
Philippines,0.000000e+00,762.114044,0 (0%),760 (1.9%)
India,0.000000e+00,587.365036,0 (0%),590 (2%)
Bangladesh,-1.907349e-12,329.917874,0 (0%),330 (2.1%)
Italy,-3.814697e-12,297.885756,0 (0%),300 (1.7%)
"Iran, Islamic Rep.",9.536743e-13,229.458025,0 (0%),230 (1.6%)
Brazil,-9.536743e-13,224.756372,0 (0%),220 (2.5%)


In [48]:
makedirs("cards/policy_tables_alt/", exist_ok=True)

for col in delta.stack().columns:
# for col in ["vp","vr"]:
    y=1e-6*delta[col]
    x=((100*rel[col][["dWtot_currency","dKtot"]]).applymap(f).astype(str)+"%"
     + ' ('+ y.applymap(f).astype(str) +' m$)'
    )


    x=x.rename(columns={"dKtot":"Asset losses label", "dWtot_currency":"Wellbeing losses  label"}
            )

    x


    y=y.rename(columns={"dKtot":"Asset losses value", "dWtot_currency":"Wellbeing losses value"}
                )

    idx = rel[col].sort_values(by="dWtot_currency", 
              ascending=False).index


    t = pd.concat([y,x],axis=1).ix[idx]
    t.index.name=policy_description[col]

    t.head(15).to_csv("cards/policy_tables_alt/"+col+".csv");
t

outputs,Asset losses value,Wellbeing losses value,Asset losses label,Wellbeing losses label
"Develop market insurance (25% of population, only nonpoor)",,,,
Senegal,-1.490116e-14,2.316111,-0% (-0 m$),3.4% (2.3 m$)
Jordan,0.000000e+00,7.393094,0% (0 m$),3.3% (7.4 m$)
Lesotho,0.000000e+00,2.599370,0% (0 m$),3.1% (2.6 m$)
Ecuador,0.000000e+00,142.305143,0% (0 m$),2.9% (140 m$)
El Salvador,-2.384186e-13,57.463758,-0% (-0 m$),2.9% (57 m$)
Sri Lanka,5.960464e-14,14.218436,0% (0 m$),2.9% (14 m$)
"Venezuela, RB",0.000000e+00,146.273569,0% (0 m$),2.7% (150 m$)
"Yemen, Rep.",-5.960464e-14,17.136597,-0% (-0 m$),2.7% (17 m$)
Togo,7.450581e-15,1.327145,0% (0 m$),2.7% (1.3 m$)


# Render cards


In [ ]:
results=pd.read_csv("results/results.csv", index_col="country")

In [ ]:
#list of policy name in apply_policy we want to use here
policy_list = ["prepare_scaleup", "borrow_abi",  "T_rebuild_K", "social_p","fap","far", "vp","kp","vr", "shew", "axfin","PDSpackage","prop_nonpoor"]
# policy_list = ["fap"]

country_list = results.dropna().index
# country_list = ["Nepal", "Honduras", "Moldova", "Rwanda"]
# "Finland" in country_list

country_list

## colors and units

In [ ]:
#list of outputs to render and color to use
colors = pd.read_csv("inputs/colors_pol_assess.csv", index_col=0)
colors

## Legend

In [ ]:
import matplotlib.patches as mpatches
from fancy_plots import savefig

patches=[]
for i in colors.index:
    patches+=[mpatches.Patch( 
                    fc=colors.ix[i,"color"],
                    ec= colors.ix[i,"edgecolor"],
                    label=colors.ix[i,"label"] ) ]
                        
fig=plt.figure()
ax=plt.gca()

l=ax.legend(handles=patches, loc="upper left", prop=font,frameon=False, title="")  
plt.setp(l.get_title(), multialignment='center')

ax.axis('off')

savefig("cards/legend_cards")


## delta

### PER COUNTRY

In [ ]:
#unit for changes   (welfare and asset loses change)
unit = dict(multiplier = 1e-6, string = "US$, millions per year")

render_pol_cards(
    delta[policy_list],
    colors,
    policy_description,
    unit,
    
#     ["China"],
  country_list,
    outfolder="cards/country/")

### per policy

In [ ]:
#unit for changes   (welfare and asset loses change)
unit = dict(multiplier = 1e-6, string = "US$, millions per year")

render_pol_cards(
    delta,
    colors,
    policy_description,
    unit,
    policy_list,
    use_title=True,
    max_lines = 15,
    outfolder="cards/policy/",
    do_by_policy=True)

## Relative to losses

### per country

In [ ]:
#unit for changes   (welfare and asset loses change)
unit = dict(multiplier = 100, string = "Percent of current losses")

render_pol_cards(
    rel[policy_list],
    colors,
    policy_description,
    unit,
    country_list, 
    name_suffix="_rel",
outfolder="cards/country/",)

### per policy

In [ ]:
#unit for changes   (welfare and asset loses change)
unit = dict(multiplier = 100, string = "Percent of current losses")

render_pol_cards(
    rel,
    colors,
    policy_description,
    unit,
    policy_list,
    name_suffix="_rel",
    outfolder="cards/policy/",
    max_lines=15,
    use_title=True,
    do_by_policy=True)

## Merge all cards to single pdf

Merges the individual province cards into one big multi page pdf. Requires ghostscript.

In [ ]:
#Country cards
province_list=glob.glob("cards/country/*.pdf")
merge_cardfiles(province_list,"cards/all_country_cards.pdf")

#Policy cards
thepollist=glob.glob("cards/policy/*.pdf")
merge_cardfiles(thepollist,"cards/all_policy_cards.pdf")

Converts the pdf files to png images. Requires Imagemagick.

In [ ]:
# convert_pdf_to_png("cards/")

# Malawi single case

In [ ]:
refout_flood = compute_resilience(macro, cat_info, hazard_ratios.query("hazard=='flood'"))

In [ ]:
m,c,hr,a,desc =apply_policy(m_,c_,hr_.query("hazard=='flood'"),a_ ,policy_name="fap")

In [ ]:
d,r,rto,nv=test_package(m,c,hr,a,return_rel=True, refout=refout_flood)

In [ ]:
x=d.ix[["Malawi","Bangladesh"],["dKtot","dWtot_currency"]]*1e-6 
x

## per capita

In [ ]:
x.T/pd.Series(dict(Malawi=.16, Bangladesh=3))

In [ ]:
#just double checking the number of people
(hazard_ratios.query("hazard=='flood' & rp==100").fa*macro["pop"]).ix[["Malawi","Bangladesh"]].mean(level="country")

## discounted

In [ ]:
x.T/pd.Series(dict(Malawi=.16, Bangladesh=3))/.06